In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import product

In [ ]:
data = pd.read_csv('PandaReach-v3_robot_data.csv')
data = data.query('Task_name != "CustomReachEnv0"')
data.head(10)

# Define functions

In [ ]:
def extract_interventions(game, divide=[]):
    groups = ['Elapsed_time']
    groups += divide
    game.loc[game['Type_of_intervention'] == 'None', 'Type_of_intervention'] = np.nan
    interventions = game.groupby(groups)['Type_of_intervention'].value_counts(dropna=False).to_frame(name='no').reset_index()
    interventions.loc[interventions['Type_of_intervention'].isna(), 'no'] = 0
    interventions.loc[interventions['Type_of_intervention'] == 'demonstration', 'no'] = 1
    interventions.loc[interventions['Type_of_intervention'] == 'feedback', 'no'] = (interventions.loc[interventions['Type_of_intervention'] == 'feedback', 'no']/5).apply(np.ceil)
    intereventions_plot = interventions.groupby(groups)['no'].sum().to_frame().reset_index()
    return intereventions_plot

In [ ]:
def state_heatmap(game):
    state_no = list(range(2, 18))
    states = product(state_no, state_no, state_no)
    hm = pd.DataFrame(states, columns=['x', 'y', 'z'])
    hm['feedback'] = 0
    hm['demo'] = 0
    for t, s in enumerate(game.State):
        x, y, z = [ int(i) for i in s.split(',')]
        e = hm.eval('x == @x and y == @y and z == @z')
        if game.loc[t, 'Type_of_intervention'] == 'feedback':
            hm.loc[e, 'feedback'] += 1

        if game.loc[t, 'Type_of_intervention'] == 'demonstration':
            hm.loc[e, 'demo'] += 1
    return hm

# Analysis

In [ ]:
data = data.reset_index()

In [ ]:
hm = state_heatmap(data)

In [ ]:
hf = hm.groupby(['x', 'y'])['feedback'].sum().to_frame().reset_index()
hd = hm.groupby(['x', 'y'])['demo'].sum().to_frame().reset_index()

In [ ]:

   
   hf = hf.pivot(index='x', columns='y', values='feedback')
hd = hd.pivot(index='x', columns='y', values='demo')

In [ ]:
plt.figure(figsize=(12, 12))
sns.heatmap(hf, annot=True, cmap='viridis', fmt='g')

In [ ]:
plt.figure(figsize=(12, 12))
sns.heatmap(hd, annot=True, cmap='viridis', fmt='g')

In [ ]:
divide = ['Task_name']
int_per_elapsed_time = extract_interventions(data, divide=divide)

In [ ]:
plt.figure(figsize=(10, 5))
sns.lineplot(data=int_per_elapsed_time, x='Elapsed_time', y='no', hue='Task_name')
int_per_elapsed_time.iloc[int_per_elapsed_time.no.argmax()]

# Questionnaire

In [ ]:
quest_data = pd.read_csv("Robot learning from rich feedback_edit2.csv")

In [ ]:
quest_data.head(10)

**Distribution of Genders by Age**

In [ ]:
age_gender_group = quest_data.groupby(['Age', 'Gender']).size().unstack()
age_gender_group.plot(kind='bar', stacked=False, width=1)
plt.xlabel('Age')
plt.ylabel('Count')
plt.title('Distribution of Genders by Age')
plt.legend(title='Gender')
plt.show()

**Pet Ownership and Training in profession**

In [ ]:
teaching_intuitive = 'I found the teaching process intuitive to me and it was easy to follow'
demo_situations = 'In which situations did you decide to provide Demonstrations?'
demo_intuitive = 'I found it intuitive provide Demonstrations.'
feedback_situations = 'In which situations did you decide to provide Evaluative Feedback?'
feedback_intuitive = 'I found it intuitive provide Evaluative Feedback.'
starting_positions_selection = 'How did you choose the starting positions of the agent (robotic arm)?'
robot_learning_effectiveness = 'The robotic arm learned effectively from my teaching inputs.'
teacher_performance = 'On a scale from 1-10 how would you rate your performance as a teacher?'
own_robotic_arm = 'Imagine you own a robotic arm, would you prefer to either personally teach it to perform and assist with everyday tasks, customizing its actions based on your preferences and needs, or have it pre-trained to assist with specific everyday tasks, ready to use right away without requiring personal teaching?'
demo_communication = 'To provide Demonstrations to a robotic arm, which way would you prefer to communicate with it?'
feedback_communication = 'To provide Evaluative Feedback to a robotic arm, which way would you prefer to communicate with it?'
age = 'Age'
gender = 'Gender'
education_level = 'What is the highest degree or level of school you have completed? If currently enrolled, highest degree received.'
pet = 'Have you ever owned a pet?'
profession_teaching = 'To what extent does your profession involve teaching or training other people?'
programming_experience = 'To what extent do you have experience with programming?'
video_games_experience = 'To what extent do you have experience playing video games?'

In [ ]:
sns.set(style="whitegrid")

In [ ]:
pet_training_group = quest_data.groupby([profession_teaching, pet]).size().unstack()
pet_training_group.plot(kind='bar')
plt.xlabel('Training')
plt.ylabel('Count')
plt.title('Relationship between Pet Ownership and Training Level')
plt.xticks(rotation=0)
plt.legend(title='Pet Ownership', labels=['Yes', 'No'])
plt.show()

In [ ]:
#BAR PLOT
plt.figure(figsize=(10, 6))
palette = {'Male': 'blue', 'Female': 'pink', 'Non binary': 'orange'}
sns.scatterplot(data=quest_data, x=education_level, y=age, hue=gender, palette=palette, s=100, alpha=0.5)
plt.xlabel('Education Level')
plt.ylabel('Age')
plt.xticks(rotation=0, size=7)
plt.title('Relationship between Age, Gender, and Education Level')
plt.legend(title='Gender')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
age_education_group = quest_data.groupby([age, education_level]).size().unstack()
age_education_group.plot(kind='bar', stacked=False, width=1)
plt.xlabel('Age')
plt.ylabel('Count')
plt.title('Distribution of Educational level by Age')
plt.legend(title='Educational Level')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(data=quest_data, x=age, y=teaching_intuitive, palette='Set1')
plt.xlabel('Age Group')
plt.ylabel('Teaching Intuitiveness')
plt.title('Teaching Intuitiveness by Age Group')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(data=quest_data, x=gender, hue=pet, palette='Set2')
plt.xlabel('Gender')
plt.ylabel('Count')
plt.title('Gender and Pet Ownership')
plt.legend(title='Pet Ownership', loc='upper right')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(data=quest_data, x=education_level, y=robot_learning_effectiveness, palette='Pastel1')
plt.xlabel('Education Level')
plt.ylabel('Robot Learning Effectiveness')
plt.title('Robot Learning Effectiveness by Education Level')
plt.xticks(size=7)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(data=quest_data, x=programming_experience, hue=age, multiple='stack', palette='Set3')
plt.xlabel('Programming Experience')
plt.ylabel('Count')
plt.title('Programming Experience by Age Group')
# plt.legend(title='Age Group')
plt.show()

# ROBOT & QUESTIONNAIRE

In [ ]:
user_data = pd.read_csv("PandaReach-v3_user_data.csv")
user_data = user_data.query('Task_name != "CustomReachEnv0"')
user_data.head(10)

In [ ]:
user_data_done_time = user_data_done_time.set_index('Participant_id')

In [ ]:
user_data['total_interactions'] = user_data.Demonstrations_used + user_data.Feedback_used
user_data.head(10)

In [ ]:
mean_interactions_participant = user_data.groupby('Participant_id').total_interactions.mean().to_frame().reset_index()

In [ ]:
total_interactions_participant = user_data.groupby('Participant_id').total_interactions.sum().to_frame().reset_index()

In [ ]:
total_completed_participant = user_data.groupby('Participant_id').Description.value_counts()

In [ ]:
mean_elapsed_time_participant = user_data.groupby('Participant_id').Elapsed_time.mean().to_frame().reset_index()

In [ ]:
total_elapsed_time_participant = user_data.groupby('Participant_id').Elapsed_time.sum().to_frame().reset_index()

In [ ]:
total_elapsed_time_participant

In [ ]:
user_data_done_time = user_data.groupby(['Participant_id'])['Description'].value_counts().to_frame(name='value')
done = user_data_done_time.query('Description == "Done"')
time = user_data_done_time.query('Description == "Time"')
user_data_done_time['Done'] = done.value
user_data_done_time['Time'] = time.value
user_data_done_time = user_data_done_time.reset_index()
user_data_done_time = user_data_done_time.drop(['Description', 'value'], axis = 1)
user_data_done_time = user_data_done_time.groupby('Participant_id').agg({
    'Done': 'first',  # Take the first non-NaN value for 'Done'
    'Time': 'first'   # Take the first non-NaN value for 'Time'
}).reset_index()

In [ ]:
user_data_done_time['Teacher'] = user_data_done_time.Done >= user_data_done_time.Time
user_data_done_time.Teacher = user_data_done_time.Teacher.map({True: 'Good', False: 'Bad'})

In [ ]:
quest_data = quest_data.set_index('Participant Number')

In [ ]:
quest_data['Teacher'] = user_data_done_time.Teacher

In [ ]:
quest_data['mean_interactions'] = mean_interactions_participant.total_interactions
quest_data['total_interactions'] = total_interactions_participant.total_interactions
quest_data['mean_elapsed_time'] = mean_elapsed_time_participant.Elapsed_time
quest_data['total_elapsed_time'] = total_elapsed_time_participant.Elapsed_time

# MEAN INTERACTION - Robot Learning Effectiveness

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(data=quest_data, x=robot_learning_effectiveness, y='mean_interactions', palette='Pastel1')
plt.xlabel('Robot Learning Effectiveness')
plt.ylabel('Mean Interactions')
plt.title('Robot Learning Effectiveness by Mean Interactions')
plt.xticks(size=7)
plt.show()

# ELAPSED TIME - AGE

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(data=quest_data, x=age, y='mean_elapsed_time')
plt.xlabel('Age')
plt.ylabel('Mean Elapsed Time')
plt.title('Age by Mean Elapsed Time')
plt.xticks(size=7)
plt.show()

# ELAPSED TIME - Programming Experience

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(data=quest_data, x=programming_experience, y='mean_elapsed_time')
plt.xlabel('Programming Experience')
plt.ylabel('Mean Elapsed Time')
plt.title('Programming Experience by Mean Elapsed Time')
plt.xticks(size=7)
plt.show()

# ELAPSED TIME - Video Games Experience

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(data=quest_data, x=video_games_experience, y='mean_elapsed_time')
plt.xlabel('Video Games Experience')
plt.ylabel('Mean Elapsed Time')
plt.title('Video Games Experience by Mean Elapsed Time')
plt.xticks(size=7)
plt.show()

# Mean Interactions - Pet

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(data=quest_data, x=pet, y='mean_interactions')
plt.xlabel('Owning a pet')
plt.ylabel('Mean Interactions')
plt.title('Owning a pet by Mean Interactions')
plt.xticks(size=7)
plt.show()

# Personally teach it - Mean Interactions

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(data=quest_data, x=own_robotic_arm, y='mean_interactions')
plt.xlabel('Owning a robotic arm')
plt.ylabel('Mean Interactions')
plt.title('Owning a robotic arm by Mean Interactions')
plt.xticks(size=7)
plt.show()

# Training - Mean Interactions

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(data=quest_data, x=profession_teaching, y='mean_interactions')
plt.xlabel('Profession Teaching')
plt.ylabel('Mean Interactions')
plt.title('Profession Teaching by Mean Interactions')
plt.xticks(size=7)
plt.show()

In [ ]:
quest_data[profession_teaching].value_counts()

# Age - Mean Interactions

In [ ]:
age_quantiles = pd.qcut(quest_data[age], q=5, duplicates='drop')

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(data=quest_data, x=age_quantiles, y='mean_interactions')
plt.xlabel('Age')
plt.ylabel('Mean Interactions')
plt.title('Age by Mean Interactions')
plt.xticks(size=7)
plt.show()

In [ ]:
age_quantiles.value_counts()

# **GENERAL STUFF**

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(data=quest_data, x='Teacher', y=teacher_performance, palette='Pastel1')
plt.xlabel('Good / Bad Teacher')
plt.ylabel('How good of a teacher did they thought they were')
plt.title('Robot Learning Effectiveness by Mean Interactions')
plt.xticks(size=7)
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(data=quest_data, x='Teacher', y='mean_interactions')
plt.xlabel('Good / Bad Teacher')
plt.ylabel('Mean Interactions')
plt.title('Good / Bad Teacher by Mean Interactions')
plt.xticks(size=7)
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(data=quest_data, x='Teacher', y='mean_elapsed_time')
plt.xlabel('Good / Bad Teacher')
plt.ylabel('Mean Elapsed Time')
plt.title('Good / Bad Teacher by Mean Elapsed Time')
plt.xticks(size=7)
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(data=quest_data, x=starting_positions_selection, y='mean_elapsed_time')
plt.xlabel('Good / Bad Teacher')
plt.ylabel('Mean Elapsed Time')
plt.title('Good / Bad Teacher by Mean Elapsed Time')
plt.xticks(size=7)
plt.show()

In [ ]:
quest_data.Teacher.value_counts()

In [ ]:
quest_data.groupby('Teacher')[starting_positions_selection].value_counts().plot(kind='bar')

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(data=quest_data, x='Teacher', y=age)
plt.xlabel('Good / Bad Teacher')
plt.ylabel('Age')
plt.title('Good / Bad Teacher by Age')
plt.xticks(size=7)
plt.show()

In [160]:
data[data['Type_of_intervention'] == 'demonstration']

,index,Participant_id,Game_name,Task_name,Game_num,Starting_position,Timestamp,Time_step,Elapsed_time,Type_of_intervention,Human_feedback,State,Action,Reward,"Q(s,a)",Advantage
587,2097,1,PandaReach-v3,CustomReachEnv1,2.0,"10,8,11",2023-08-07 22:59:08.256295,676.0,58.0,demonstration,none,"9,4,9",4.0,-1.0,-0.548281,1.548281
588,2098,1,PandaReach-v3,CustomReachEnv1,2.0,"10,8,11",2023-08-07 22:59:08.761404,689.0,59.0,demonstration,none,"10,5,9",4.0,-1.0,-0.857825,1.857825
589,2099,1,PandaReach-v3,CustomReachEnv1,2.0,"10,8,11",2023-08-07 22:59:09.039373,696.0,59.0,demonstration,none,"10,5,9",4.0,-1.0,-0.528912,1.528912
590,2100,1,PandaReach-v3,CustomReachEnv1,2.0,"10,8,11",2023-08-07 22:59:09.661600,712.0,59.0,demonstration,none,"10,5,9",4.0,-1.0,-0.364456,1.364456
591,2101,1,PandaReach-v3,CustomReachEnv1,2.0,"10,8,11",2023-08-07 22:59:10.840359,742.0,61.0,demonstration,none,"10,5,9",1.0,-1.0,-1.489954,1.125498
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127941,148843,38,PandaReach-v3,CustomReachEnv3,1.0,"6,10,10",2023-08-10 16:29:54.273715,2037.0,146.0,demonstration,none,"8,9,11",1.0,-0.5,0.350000,0.650000
127942,148844,38,PandaReach-v3,CustomReachEnv3,1.0,"6,10,10",2023-08-10 16:29:54.728281,2045.0,146.0,demonstration,none,"8,9,11",1.0,-0.5,0.325000,0.675000
127943,148845,38,PandaReach-v3,CustomReachEnv3,1.0,"6,10,10",2023-08-10 16:30:01.513622,2158.0,153.0,demonstration,none,"9,9,11",2.0,-0.5,0.650000,0.350000
127944,148846,38,PandaReach-v3,CustomReachEnv3,1.0,"6,10,10",2023-08-10 16:30:01.819464,2163.0,153.0,demonstration,none,"8,9,11",2.0,-0.5,0.650000,0.350000


In [164]:
demonstration_df = data[data['Type_of_intervention'] == 'demonstration']
first_demo_times = demonstration_df.groupby(['Participant_id', 'Task_name', 'Game_num'])['Elapsed_time'].min().reset_index()
first_demo_times.columns = ['Participant_id', 'Task_name', 'Game_num', 'First_demo_time']

first_demo_times

,Participant_id,Task_name,Game_num,First_demo_time
0,1,CustomReachEnv1,2.0,58.0
1,1,CustomReachEnv2,1.0,46.0
2,1,CustomReachEnv3,1.0,48.0
3,2,CustomReachEnv1,1.0,48.0
4,2,CustomReachEnv1,2.0,50.0
...,...,...,...,...
144,37,CustomReachEnv2,1.0,93.0
145,38,CustomReachEnv1,1.0,46.0
146,38,CustomReachEnv2,1.0,81.0
147,38,CustomReachEnv2,3.0,15.0


In [177]:
feedback_df = data[data['Type_of_intervention'] == 'feedback']
first_feedback_times = feedback_df.groupby(['Participant_id', 'Task_name', 'Game_num'])['Elapsed_time'].min().reset_index()
first_feedback_times.columns = ['Participant_id', 'Task_name', 'Game_num', 'First_demo_time']

first_feedback_times.groupby(['Task_name'])[['Participant_id', 'First_demo_time']].mean().reset_index()

,Task_name,Participant_id,First_demo_time
0,CustomReachEnv1,18.545455,43.386364
1,CustomReachEnv2,19.275000,42.550000
2,CustomReachEnv3,17.452381,53.071429
